# My Update
In this notebook I will expand the functionalities of the current RAG system implemented together with my group.

In [1]:
import json

from rag_service.rag_chat import rag_interaction

## Knowledge Base
Instead of getting the Knowledge Base data through the API, as we did for the project, I downloaded the data so I could have it locally and not have to log in every time to get it. The data in the KB consists of two JSON files that contain the description of the available KPIs and the list of machines with the corresponding KPIs.

In [2]:
file_path = "data/documentation.txt"

with open(file_path, "r") as file:
    documentation = file.read()

In [3]:
file_path = "data/kpis.json"

with open(file_path, "r") as file:
    kpi_data = json.load(file)

In [4]:
file_path = "data/machines.json"

with open(file_path, "r") as file:
    machine_data = json.load(file)

In [5]:
data = [kpi_data, machine_data]
data.append({"type": "txt", "content": documentation})

## Queries

In [6]:
query = "How many machines are there?"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
print(response)

There are 16 machines in total.


---

In [8]:
query = "How many KPIs are there?"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
print(response)

There are 14 KPIs.


---

In [10]:
query = "Are there KPIs related to consumption?"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
print(response)

Yes, there are KPIs related to consumption. They include "consumption", "consumption_working", and "consumption_idle".


---

In [12]:
query = "Can you speak italian?"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
print(response)

No, I am not capable of speaking Italian or any other language. My capabilities are limited to generating text based on the provided knowledge and context. I do not possess the ability to engage in real-time conversations or express myself through spoken languages.


---

In [21]:
query = "Show me the list of the KPIs available for the Assembly Machine 1"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
print(response)

Here are the KPIs available for the Assembly Machine 1:

working_time, idle_time, offline_time, average_cycle_time, cost, cost_working, cost_idle, good_cycles, bad_cycles


---

In [16]:
query = "Generate a new KPI"
response = rag_interaction(data, query)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
print(response)

{
  "KPIs": [
    {
      "name": "Total Idle Time",
      "type": "Consumption",
      "description": "The total time spent idle by all machines",
      "unit_of_measure": "Minutes",
      "formula": "(consumption_idle * (machines.length / 16))"
    }
  ]
}


## Report Generation

In [18]:
file_path = "data/report_data.json"

with open(file_path, "r") as file:
    report_data = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'data/report_data.json'

In [ ]:
response = rag_interaction(report_data, generate_report=True)

In [ ]:
print(response)

## Exploit the Memory Functionality

### Dynamic Reports

### Error Correction

### Personalized Recommendations